# HARDI Deterministic Tractography

So far we have learnt how to perform tractography using the DTI model to estimate white matter (WM) fibre orientations. While DTI is still widely used due to its simplicity and speed, its limitations in resolving complex fiber architecture make it less suitable for detailed studies in regions with crossing fibers. As an alternative, High Angular Resolution Diffusion Imaging (HARDI) tractography offers several advantages over DTI tractography, especially in the context of mapping complex white matter structures in the brain. HARDI is characterized by the use of a high number of diffusion gradient directions (typically 30 or more) and moderate to high b-values (commonly between 1000 and 3000 s/mm²). These parameters allow for a more detailed exploration of the diffusion characteristics of tissues, particularly in regions of the brain where multiple fiber populations intersect.

## DTI vs HARDI 

### Ability to Resolve Crossing Fibers

DTI: DTI is limited by its assumption that diffusion is Gaussian and can be fully characterized by a single diffusion tensor. This means DTI can only resolve one dominant fiber direction per voxel. In regions with complex fiber architecture (e.g., where fibers cross, fan out, or merge), DTI fails to accurately represent the underlying anatomy, often resulting in misleading or incomplete tractography.

HARDI: HARDI can distinguish multiple fiber orientations within a single voxel. This is achieved by acquiring diffusion-weighted images along a larger number of gradient directions and multible b-values, which provides a more detailed sampling of the diffusion profile.


### Improved Fiber Tracking in Complex Brain Regions

DTI: In areas with crossing fibers, DTI’s limitations can lead to erroneous fiber tracking, where the tractography may either fail to follow certain tracts or incorrectly track fibers in the wrong direction.
    
HARDI: Because HARDI can resolve multiple fiber directions, it provides a more accurate representation of white matter architecture, especially in areas like the centrum semiovale, where up to 90% of the voxels contain crossing fibers. This leads to more accurate tractography in these regions, reducing the likelihood of false negatives (missing tracts) and false positives (incorrect tracts).

    
### More Accurate Measurement of Anisotropy

DTI: DTI’s single tensor model often leads to oversimplified measures of anisotropy, which may not accurately reflect the true diffusion properties in voxels with more than one dominant fiber direction.
        
HARDI: HARDI models the diffusion signal in a way that can better capture the distribution of diffusion directions within a voxel. This allows for a more accurate estimation of anisotropy in voxels with complex fiber configurations. Metrics derived from HARDI, like Generalized Fractional Anisotropy (GFA), provide a more nuanced understanding of diffusion patterns compared to Fractional Anisotropy (FA) from DTI.


### Advanced Tractography Algorithms

DTI: Traditional DTI tractography algorithms, are limited by their ability to handle only a single fiber direction per voxel, which can result in less accurate fiber reconstructions.
        
HARDI: HARDI enables the use of more sophisticated tractography algorithms (e.g. Constrained Spherical Deconvolution), which can better handle the complexities of brain fiber architecture. These algorithms take into account the multiple fiber orientations per voxel, improving the accuracy and robustness of fiber tracking.


## HARDI Tractography using Constrained Spherical Deconvolution (CSD)

Constrained Spherical Deconvolution (CSD) is an advanced technique used in diffusion MRI to model and estimate the distribution of fiber orientations within each voxel of the brain. It is particularly effective for resolving complex white matter architecture, such as crossing, kissing, or fanning fibers, which are common in the human brain. The Fiber Orientation Distribution (FOD) is a function that represents the distribution of fiber orientations within a voxel. 

### Key steps in CSD reconstruction

1. Acquisition of dMRI Data: HARDI data is required for CSD modelling (usually more than 30 gradient directions and often 60 or more). Acquiring data for multiple b-value shells cans also improve accuracy. 

2. Response Function Estimation: A response function, which models the diffusion signal for a single fiber orientation, is estimated from the data. This function is usually derived from regions of the brain known to contain coherently aligned fibers, such as the corpus callosum.

3. Spherical Deconvolution Process: The measured dMRI signal in each voxel is deconvolved using the response function to estimate the FOD. In image and signal processing, deconvolution is the process of reversing the effects of convolution, which in this context refers to how the measured diffusion MRI signal is blurred or spread out due to the presence of multiple fibers. This process effectively "sharpens" the diffusion signal, revealing multiple fiber orientations within the voxel.

4. Application of Constraints: Non-negativity constraints are applied to ensure that the resulting FOD does not predict negative fiber densities, which would be unphysical. Regularization techniques, such as lasso or ridge regression, may also be used to stabilize the solution and reduce noise.


### Limitations of CSD

1. Requires High-Quality Data: CSD relies on HARDI data with sufficient signal-to-noise ratio, which can be more demanding in terms of acquisition time and data quality compared to simpler diffusion models.

2. Computationally Intensive: The deconvolution and regularization processes are computationally demanding, requiring significant processing power and time, especially for whole-brain analyses.

3. Sensitive to Model Assumptions: The accuracy of CSD depends on the correct estimation of the response function and the choice of regularization parameters. Incorrect assumptions can lead to artifacts or inaccurate fiber orientation distributions.

This tutorial will cover how to use dipy tools to perform deterministic and probabilistic tractography with CSD modelling and it will include examples from https://docs.dipy.org/1.11.0/examples_built/fiber_tracking/. 


## Deterministic Tractography using CSD

We will start by loading the pre-processed diffusion MRI data:

In [ ]:
#load general modules
import os
import numpy as np
import nibabel as nib

#load dipy modules
from dipy.core.gradients import gradient_table
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti, save_nifti

#load modules for visualization
import matplotlib.pyplot as plt

#define the paths to the data
scripts_dir = os.getcwd()
bids_dir = "../data/bids"
out_dir = "../data/derivatives"

#subject code to be used in this example
sub='01'

#load the pre-processed data
data_preproc, affine, data_preproc_img = load_nifti(f"{out_dir}/preprocessing/sub-{sub}/eddy_unwarped_images.nii.gz", return_img=True)

#load the bvals and bvecs
bvals, bvecs = read_bvals_bvecs(f"{bids_dir}/sub-{sub}/dwi/sub-{sub}_acq-AP_dwi.bval", f"{out_dir}/preprocessing/sub-{sub}/eddy_unwarped_images.eddy_rotated_bvecs")

#load the gradient table
gtab = gradient_table(bvals, bvecs)


#load the binary brain mask
brain_mask, affine_mask = load_nifti(f"{out_dir}/preprocessing/sub-{sub}/hifi_nodif_brain_mask.nii.gz")
masked_data = data_preproc * brain_mask[..., np.newaxis]

#create a basic white matter mask by thresholding the FA map
#load FA map (from Tutorial 3)
data_FA, affine_FA = load_nifti(f"{out_dir}/DTI/sub-{sub}/dti_fa.nii.gz")
white_matter_mask = (data_FA > 0.2).astype(int)

### Step 1: Response Function Estimation

Dipy offers various methods to compute CSD response functions (see https://workshop.dipy.org/documentation/1.6.0./examples_built/reconst_csd/). For this tutorial, we will use dipy's "auto_response_ssst" function to estimate the response function from a region of interest located in the center of the brain.

In [ ]:
from dipy.reconst.csdeconv import auto_response_ssst

#estimate response function
response, ratio = auto_response_ssst(gtab, data_preproc, roi_radii=10, fa_thr=0.7)


The response tuple contains two elements. The first is an array with the eigenvalues of the response function and the second is the average S0 for this response.

It is good practice to always validate the result of auto_response_ssst. For this purpose we can print the elements of response and have a look at their values.

In [ ]:
print(response)

The tensor generated from the response must be prolate (two smaller eigenvalues should be equal) and look anisotropic with a ratio of second to first eigenvalue of about 0.2. Or in other words, the axial diffusivity of this tensor should be around 5 times larger than the radial diffusivity.

In [ ]:
print(ratio)

We can double-check that we have a good response function by visualizing the response function’s ODF:

In [ ]:
from dipy.viz import window, actor
from dipy.sims.voxel import single_tensor_odf
from dipy.data import default_sphere

# Enables/disables interactive visualization
interactive = True

scene = window.Scene()
evals = response[0]
evecs = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]]).T


response_odf = single_tensor_odf(default_sphere.vertices, evals, evecs)
# transform our data from 1D to 4D
response_odf = response_odf[None, None, None, :]
response_actor = actor.odf_slicer(response_odf, sphere=default_sphere,
                                  colormap='plasma')
scene.add(response_actor)
print('Saving illustration as csd_response.png')
window.record(scene, out_path='csd_response.png', size=(200, 200))
if interactive:
    window.show(scene)

### Step 2: Define the CSD model

The next step is to define the CSD model according to the acquisition parameters estimated from the GradientTable:

In [ ]:
from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel

#define and fit the CSD model to the data
csd_model = ConstrainedSphericalDeconvModel(gtab, response, sh_order_max=6)
csd_fit = csd_model.fit(data_preproc, mask=white_matter_mask)


For quality assurance lets visualize the FODs for a data slice:

In [ ]:
from dipy.data import get_sphere

#sphere = get_sphere('repulsion724')
sphere = get_sphere('symmetric362')

scene = window.Scene()
odf_actor = actor.odf_slicer(csd_fit.odf(sphere), sphere=sphere, scale=0.5,
                             colormap='plasma')
scene.add(odf_actor)
print('Saving illustration as csd_fodfs.png')
window.record(scene, n_frames=1, out_path='csd_fodfs.png', size=(600, 600))
window.show(scene)


Like we did for the DTI model, directions from the CSD model can be extracted using dipy's function "peaks_from_model". For the CSD model, the inputs "sphere", "relative_peak_threshold" and "min_separation_angle" become relevant.

For details of the inputs function, lets run "help(peaks_from_model)".


In [ ]:
from dipy.direction import peaks_from_model

help(peaks_from_model)

We will run the "peaks_from_model" function using the default discrete directions provided by dipy ("default_sphere"), a relative peak threshold of 0.5 and a minimum angular distance between peaks of 45 degrees:

In [ ]:
from dipy.data import default_sphere
csd_peaks = peaks_from_model(csd_model, masked_data, sphere=default_sphere,
                             relative_peak_threshold=0.5,
                             min_separation_angle=45,
                             mask=white_matter_mask)

For quality assurance we can visualize a slice from the direction field which we will use to perform the tracking.

In [ ]:
from dipy.viz import window, actor, has_fury

if has_fury:
    scene = window.Scene()
    scene.add(actor.peak_slicer(csd_peaks.peak_dirs,
                                csd_peaks.peak_values,
                                colors=None))

    window.record(scene, out_path='csd_direction_field.png', size=(900, 900))
    window.show(scene, size=(800, 800))


### Step 3: Define the seeds

As we did for the DTI example, we will place a 2 × 2 × 2 grid of seeds per voxel, in a sagittal slice of the corpus callosum.


In [ ]:
from dipy.tracking import utils

#load corpus callosum mask
cc_mask, affine_cc = load_nifti(f"{out_dir}/DTI/sub-{sub}/CC_mask.nii.gz")

seed_mask = (cc_mask == 1)
seeds = utils.seeds_from_mask(seed_mask, affine_cc, density=[2, 2, 2])

### Step 4: Define the tracking stopping criteria

For this tutorial we will use the general fractional anisotropu (GFA) to define the stopping criteria. Please see the sipy documentation for other examples of stopping criteria. https://workshop.dipy.org/documentation/1.7.0/examples_built/13_fiber_tracking/tracking_stopping_criterion/


In [ ]:
from dipy.reconst.shm import CsaOdfModel
from dipy.tracking.stopping_criterion import ThresholdStoppingCriterion

csa_model = CsaOdfModel(gtab, sh_order_max=6)
gfa = csa_model.fit(data_preproc, mask=white_matter_mask).gfa
stopping_criterion = ThresholdStoppingCriterion(gfa, 0.2)

### Step 5: Propagating the streamlines

We now have everything we meed to generate the streamlines. As in previous tutorials, the streamline generation object must be initialised:


In [ ]:
from dipy.tracking.local_tracking import LocalTracking

streamlines_generator = LocalTracking(csd_peaks, stopping_criterion, seeds,
                                      affine=affine, step_size=0.5)

And finally we generate the strealines using the "Streamline" function from "dipy.tracking.streamline" and save the tractogram as a Trackvis file:

In [ ]:
from dipy.tracking.streamline import Streamlines
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import save_trk

# generate the streamlines 
streamlines = Streamlines(streamlines_generator)

#save the streamlines
sft = StatefulTractogram(streamlines, data_preproc_img, Space.RASMM)
save_trk(sft, f"{out_dir}/CSD/sub-{sub}/CSD_CC_deterministic_tract.trk", streamlines)

We can also display the resulting streamlines using the fury python package:

In [ ]:
from dipy.viz import colormap

if has_fury:
    # Prepare the display objects.
    color = colormap.line_colors(streamlines)

    streamlines_actor = actor.line(streamlines,
                                   colormap.line_colors(streamlines))

    # Create the 3D display.
    scene = window.Scene()
    scene.add(streamlines_actor)

    # Save still images for this static example. Or for interactivity use
    window.record(scene, out_path='csd_tractogram_determ.png', size=(800, 800))
    window.show(scene)

### Probabilistic Tractography using CSD

For probabilistic tractography with CSD we can re-use the response functions, seeds and stopping criteria as set up above for deterministic tractography. The main difference is that we will use the discrete FOD for sampling tracking directions using "ProbabilisticDirectionGetter".  Firstly, we need to clip the FOD because in order for it to be used as a probability mass fucntion (PMF) to sample directions it cannot have negative values. Ideally, the FOD should already be positive, but because of noise and/or model failures sometimes it can have negative values.

In [ ]:
from dipy.direction import ProbabilisticDirectionGetter
from dipy.data import get_sphere

#load the directions evenly distributed on a sphere
sphere = get_sphere('symmetric724')

# define the PMF from the ODF
fod = csd_fit.odf(sphere)
pmf = fod.clip(min=0)

# instanciate the probabilistic direction getter
prob_dg = ProbabilisticDirectionGetter.from_pmf(pmf, max_angle=30.,
                                                sphere=sphere)

# initialise the streamlines generator
streamline_generator_prob = LocalTracking(prob_dg, stopping_criterion, seeds,
                                     affine, step_size=0.5)

# generate the streamlines
streamlines_prob = Streamlines(streamline_generator_prob)

#save the streamlines
sft_prob = StatefulTractogram(streamlines_prob, data_preproc_img, Space.RASMM)
save_trk(sft_prob, f"{out_dir}/CSD/sub-{sub}/CSD_CC_probabilistic_tract.trk", streamlines_prob)

Finally, we can display the resultiing probabilistic tractogram: 

In [ ]:
from dipy.viz import colormap

if has_fury:
    # Prepare the display objects.
    color_prob = colormap.line_colors(streamlines_prob)

    streamlines_actor_prob = actor.line(streamlines_prob,
                                   colormap.line_colors(streamlines_prob))

    # Create the 3D display.
    scene = window.Scene()
    scene.add(streamlines_actor_prob)

    # Save still images for this static example. Or for interactivity use
    window.record(scene, out_path='csd_tractogram_prob.png', size=(800, 800))
    window.show(scene)